datetime
sortolas hogy latszodjon milyen idointervallum az adat +head,tail
utana hogy milyen weather a realis
filterezes: ahol min > max
ahol trip station name rossz
kitolteni a hianyzo zip code-ot station name alapjan
atteni stringrol int-re, hogy joinolni lehessen
outlierezes (boxplot)

In [ ]:
### majd ez alapjan kell joinilni
landmark_dict = {94107:'San Francisco',
                 94063:'Redwood City',
                 94301:'Palo Alto', 
                 94041:'Mountain View', 
                 95113:'San Jose'}
weather_df['landmark'] = weather_df['zip'].map(lambda x: landmark_dict[x])

## Preparing for joining the dataframes

- weather and trip can be joined via date and zip code
- weather and trip can be joined to station via finding the landmark name of the zip codes

We'll go through these columns and transform them so the dataframes can be joined.

### 1 - exercise
Load the datasets into pandas dataframes called trip, weather and station.<br><br>
Let's start with the date columns! <br>
In the trip dataframe create two new columns called 'start_datetime' and 'end_datetime'. These should contain the values from column 'Start Date' and 'End date' converted into datetime. <br><br>
In the weather dataframe create a new column called 'datetime' similarly from the 'Date' column.<br><br>
Display the datatypes of these columns to check the difference between the old and the new format.

In [15]:
### Your code here
import pandas as pd

weather = pd.read_csv('weather_filled.csv')
trip = pd.read_csv('trip_filled.csv')
station = pd.read_csv('station.csv')

trip['start_date'] = pd.to_datetime(trip['Start Date'], format='%m/%d/%Y %H:%M')
trip['end_date'] = pd.to_datetime(trip['End Date'], format='%m/%d/%Y %H:%M')
weather['datetime'] = pd.to_datetime(weather['Date'], format='%m/%d/%Y')

/home/esztersomos/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1 - check yourself

In [2]:
if (trip['Start Date'] != trip['start_date'].dt.strftime('%-m/%-d/%Y %-H:%M')).sum() == 0:
    print('start_date column is successfully converted')
else:
    print('start_date column is NOT successfully converted')
if (trip['End Date'] != trip['end_date'].dt.strftime('%-m/%-d/%Y %-H:%M')).sum() == 0:
    print('end_date column is successfully converted')
else:
    print('end_date column is NOT successfully converted')
if (weather['Date'] != weather['datetime'].dt.strftime('%-m/%-d/%Y')).sum() == 0:
    print('datetime column is successfully converted')   
else:
    print('datetime column is NOT successfully converted')

start_date column is successfully converted
end_date column is successfully converted
datetime column is successfully converted


### 2 - exercise

Now, that we can use the date values in the weather and the trip dataframe, let's check if they cover the same interval! <br>
Create four variable called weather_datetime_min, weather_datetime_max, trip_start_date_min, trip_start_date_max, trip_end_date_min and trip_end_date_max containing the minimum and maximum values from the named columns.<br>
Display them to see if they the two dataframes have the same minimum and maximum date!

In [35]:
weather_datetime_min = weather['datetime'].min()
weather_datetime_max = weather['datetime'].max()
trip_start_date_min = trip['start_date'].min()
trip_start_date_max = trip['start_date'].max()
trip_end_date_min = trip['end_date'].min()
trip_end_date_max = trip['end_date'].max()

### 2 - check yourself

In [43]:
result_dict = {'weather_datetime_min':pd.Timestamp('2013-08-29 00:00:00'),
              'weather_datetime_max':pd.Timestamp('2014-02-28 00:00:00'),
              'trip_start_date_min':pd.Timestamp('2013-08-29 09:08:00'),
              'trip_start_date_max':pd.Timestamp('2023-08-29 14:13:00'),
              'trip_end_date_min':pd.Timestamp('2013-08-29 09:11:00'),
              'trip_end_date_max':pd.Timestamp('2014-03-02 16:52:00')}
for k,v in result_dict.items():
    if eval(k) == v:
        print('{} is correct'.format(k))
    else:
        print('{} is NOT correct'.format(k))

weather_datetime_min is correct
weather_datetime_max is correct
trip_start_date_min is correct
trip_start_date_max is correct
trip_end_date_min is correct
trip_end_date_max is correct


### 3 - exercise

So it looks like that there are some rows with invalid start_date in the trip dataframe. <br>
Let's see, in how many rows is the start_date later than the latest end_date. You can use the trip_end_date_max variable you've created in the previous task. <br>
Save the number of rows into a variable called nb_not_valid_start_date_rows.

In [41]:
nb_not_valid_start_date_rows = len(trip[trip.start_date>trip_end_date_max])

### 3 - check yourself

In [44]:
if nb_not_valid_start_date_rows == 20:
    print('nb_not_valid_start_date_rows is correct')
else:
    print('nb_not_valid_start_date_rows is NOT correct')

nb_not_valid_start_date_rows is correct


### 4 - exercise

As the number of row with invalid start_date is not high, let's just drop them. <br>
Create a new dataframe called trip_valid_start where these rows are dropped.

In [95]:
trip_valid_start = trip[trip.start_date<=trip_end_date_max].copy()

### 4 - check yourself

In [49]:
if len(trip_valid_start) == 143985 and trip_valid_start['start_date'].max()==pd.Timestamp('2014-02-28 23:20:00'):
    print('New dataframe is correct')
else:
    print('Something is not correct with the new dataframe')

New dataframe is correct


### 5 - exercise
In the weather dataframe the datetime refers to a day, while in the trip_valid_start dataframe the dates refer to exact times. <br>
For being able to join, we'll need to create new columns in both dataframes with only the date part of the timestamps. In the trip_valid_start dataframe we'll use the start_date to join on. <br>
In both dataframes create column called date_to_join in which the datetime/start_date column is transformed to datetime.date format! <br>
Using set().intersection and set().difference see how many common and different values are in the date_to_join columns in the two dataframes. Create variables called nb_common and nb_different and save the result into them!

In [96]:
weather['date_to_join'] = weather['datetime'].dt.date
trip_valid_start['date_to_join'] = trip_valid_start['start_date'].dt.date

In [100]:
nb_common = len(set(trip_valid_start.date_to_join.unique()).intersection(set(weather.date_to_join.unique())))
nb_different = len(set(trip_valid_start.date_to_join.tolist()).difference(set(weather.date_to_join.tolist())))

### 5 - check yourself

In [102]:
if nb_common == 184 and nb_different==0:
    print('The date_to_join columns are correct')
else:
    print('Something is not correct')

The date_to_join columns are correct


### 6 -exercise

The next column we'll use to join dataframes is the zip code

### todo ezt majd az api/scrapingnel, eloszor a station landmarkhoz kell zipcode lista, igy mehet a weather zip -> landmarkba, es a trip Zip Code is landmarkba. Viszont trip Zip Code vegyesen float es string

In [123]:
station.station_id.unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 21, 22, 23,
       24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41,
       42, 45, 46, 47, 48, 49, 50, 51, 54, 55, 56, 57, 58, 59, 60, 61, 62,
       63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 80, 82,
       83])

In [125]:
trip['Start Terminal'].unique()

array([66, 10, 27, 59,  4,  8, 49,  6, 28, 64, 41, 47, 67, 74, 39, 22, 58,
       56,  9, 55,  3, 26, 61, 72, 45, 76, 62, 46, 35, 68,  5, 71, 69, 73,
       48, 75, 70, 11, 50, 77, 65, 60, 30, 42, 34, 54, 57, 23, 37, 51, 13,
       63,  2,  7, 36, 29, 33, 14, 12, 21, 38, 24, 16, 25, 80, 32, 31, 82,
       83])

In [126]:
trip['End Terminal'].unique()

array([66, 10, 27, 67, 59,  5,  8, 11, 54,  4, 28, 64, 56, 47, 50, 76, 22,
       58, 77, 74,  9, 51, 25, 72, 45, 61, 62, 46, 41, 37, 57,  3, 69, 65,
       12, 70, 42, 71, 55, 48, 35, 23, 39, 49, 63,  6, 26, 75, 73,  2, 34,
       68,  7, 60, 13, 36, 21, 33, 14, 29, 38, 30, 16, 24, 80, 31, 32, 82,
       83])